## Infix, Postfix and Prefix

There are 3 main ways by which we can write an expression and evaluate it.

These are :

- Infix Notation
    
    This is the normal way in which we write expressions normally.
    
    For example a+b-c. In infix we can use parentheses to override operator precedence and associativity.
    
    It is generally hard to evaulate and expression written in this notation cannot be evaluated in 1 pass.

- Postfix Notation 
    
    In postfix notation, operators appear after operands.
    
    For example, a+b-c will be ab+c- in postfix notation. No parentheses are required as when evaluating, order is automatically detected.
    
    Expression can be evaluated in a single pass.

- Prefix Notation
    
    In prefix notation, operators appear before operands.
    
    For example, a+b-c will be -+abc in prefix. No parentheses are required here during evaluation.
    
    Expressions can be evaluated in a single pass.

In [2]:
operators = ["/","*","-","+","^"]
def check_precedence(a, b):
    # Only few operators are used
    levels = {
        "^": 1,
        "/": 2,
        "*": 2,
        "+": 3,
        "-": 3
    }
    return levels[a] - levels[b]


### Infix to Postfix Conversion

#### Naive Approach

Naive approach is simpe. We put parentheses on each expression according to associativity and precedence and then solve brackets first.

It is easier for humans to understand but harder to implement on computers.

Example : infix -> a+b-c

    Here precedence of + and - are same so we will put brackets according to associativity. + and - are evaluated from left to right.

    Our resulting equation will be ((a+b)-c). After solving inner parentheses, ((ab+)-c). 

    Solving outer bracket, we get ab+c-.

#### Efficient Approach

We will use stack to efficiently convert infix to postfix.

Our algorithm is :


```python
- Create a stack for string values.
- Start looping through the string.
- if an operand is encountered, output it.
- if opening parentheses is encountered, push it to stack.
- if closing stack is encountered, pop values from stack and output them till closing counterpart is encountered.
- if an operator is encountered, 3 cases arise :
    - if precedence of operator on top of stack is lower than current operator, push current operator.
    - if precedence of operator on top of stack is higher than current operator, pop and output operators till oprerator of lower precedence is found or stack is empty.
    - if precedence is equal, associativity is used, for example if left to right associativity is used, operator appearing before current operator has higher precedence than current operator, so do as done for top higher precedence.
- After loop completes pop and output all remaining operators
```

##### Implementation

In [47]:
def infix_to_postfix(expression) -> str:
    st = []
    res = ""
    for i in expression:
        if i == "(":
            st.append(i)
        elif i == ")":
            x = st.pop()
            while x != "(":
                res += x
                x = st.pop()
        elif i in operators:
            if len(st) == 0 or all([i not in operators for i in st]):
                st.append(i)
            else:
                top = st[-1]
                if top not in operators:
                    st.append(i)
                    continue
                precedence = check_precedence(i, top)
                if precedence < 0:
                    st.append(i)
                else:
                    while st and precedence >= 0:
                        res += st.pop()
                        if st:
                            precedence = check_precedence(i, st[-1])
                    st.append(i)
        else:
            res += i
    while st:
        res += st.pop()

    return res


In [48]:
print(infix_to_postfix("a+b-c"))
print(infix_to_postfix("((A*B)+(C/D))"))

ab+c-
AB*CD/+


### Evaluation of postfix

Postfix expressions can be evaluated by using stack. 

Here we will use a single stack.

#### Algorithm

```python
- Create a stack.
- Start looping through the postfix exression.
- If operand is encountered, push it to stack.
- If operator is encountered, pop last 2 operands, A=Top, B=Operator next to top.
    - Evaluate B operator A.
    - Push result to stack.
- At the end, a single result will be left, pop it.
```

#### Implementation

In [22]:
def evaluate_postfix(expression) :
    st = []
    if " " in expression:
        expression = expression.split()
    for i in expression :
        if i in operators :
            a = st.pop()
            b = st.pop()
            st.append(eval(f"{b} {i} {a}"))
        else :
            st.append(i)
    return st.pop()

In [23]:
print(evaluate_postfix("562-+")) # abc+-
print(evaluate_postfix("43*63/+")) # AB*CD/+
# No support for multivalued numbers eg: 43*123/+ will result in something weird
# Provide space separated values if multivalued support is wanted.
print(evaluate_postfix("4 3 * 12 3 / +")) # AB*CD/+

9
14.0
16.0


### Infix to Prefix

#### Naive Approach

Just like infix to postfix, we first fully parenthesis the expression and then start converting from innermost level.

#### Efficient Approach

We will use stack for this approach. Here a difference is that we get reverse of result by using algorithm. Thus we have to reverese result of algorithm to get correct result. 

Our algorithm is :


```python
- Create a stack for string values.
- Start looping through the string in reverse order.
- if an operand is encountered, output it.
- if closing parentheses is encountered, push it to stack.
- if opening stack is encountered, pop values from stack and output them till closing counterpart is encountered.
- if an operator is encountered, 3 cases arise :
    - if precedence of operator on top of stack is lower than current operator, push current operator.
    - if precedence of operator on top of stack is higher than current operator, pop and output operators till oprerator of lower precedence is found or stack is empty.
    - if precedence is equal, associativity is used, for example if left to right associativity is used, operator appearing before current operator has higher precedence than current operator, so do as done for top higher precedence.
- After loop completes pop and output all remaining operators
- Reverse the prefix string formed.
```

One other technique is to apply postfix conversion on reverse of infix expression and reversing whole result. Only applies if no brackets present.

##### Implementation

In [72]:
def infix_to_prefix(expression) -> str:
    st = []
    res = ""
    for i in expression[::-1]:
        if i == ")":
            st.append(i)
        elif i == "(":
            x = st.pop()
            while x != ")":
                res += x
                x = st.pop()
        elif i in operators:
            if len(st) == 0 or all([i not in operators for i in st]):
                st.append(i)
            else:
                top = st[-1]
                if top not in operators:
                    st.append(i)
                    continue
                precedence = check_precedence(i, top)
                if precedence < 0:
                    st.append(i)
                else:
                    while st and precedence >= 0:
                        res += st.pop()
                        if st:
                            precedence = check_precedence(i, st[-1])
                    st.append(i)
        else:
            res += i
    while st:
        res += st.pop()

    return res[::-1]

In [73]:
print(infix_to_prefix("(x+(y*(z-w)))"))
print(infix_to_prefix("a+b-c"))
print(infix_to_postfix("a+b-c"[::-1])[::-1])
print(infix_to_prefix("x^y^z"))

+x*y-zw
+a-bc
+a-bc
^x^yz


### Evaluation of prefix expression

Evaluation of prefix is same as postfix with minor changes. The changes are :

- Traverse expression from right to left instead of left to right.
- Apply operator as A operator B where A is top and B is element next to top.
  

#### Implementation

In [74]:
def evaluate_prefix(expression) :
    expression = expression[::-1]
    st = []
    for i in expression :
        if i in operators : 
            a = st.pop()
            b = st.pop()
            st.append(eval(f"{a} {i} {b}"))
        else :
            st.append(i)
    return st.pop()

In [75]:
print(evaluate_prefix("+2/4*5^-53^54"))
print(evaluate_prefix("+4-41"))

2.2666666666666666
7
